In [5]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search, Q

from collections import defaultdict
from datetime import timedelta, date, datetime
import json


In [7]:
data_file_names = ['grossveranstaltungen', 'sport-fussball',
              'unfaelle', 'demonstrationen',
              'immigration',  'straftaten-anschlaege'
                  ]

In [9]:
data = {}
for name in data_file_names:
    with open("data/"+name+".json") as data_file:
        topic_data = json.load(data_file)
        data[name] = topic_data

In [15]:
data['demonstrationen']['data'][0]['articles'][0]

{'body': 'Am Vorabend des G20-Gipfels in Hamburg ist es wie befürchtet zu massiven Auseinandersetzungen zwischen Polizei und Demonstranten gekommen. Bei der "Welcome to Hell"-Demonstration gegen das Treffen flogen am Donnerstagabend Flaschen, Feuerwerkskörper wurden gezündet, später brannten umgestürzte Mülltonnen und Autos. Die Polizei stürmte in den Demonstrationszug, setzte wiederholt Wasserwerfer und Pfefferspray ein und trieb die rund 12.000 Teilnehmer auseinander. Die Polizei meldete mindestens 111 verletzte Beamte. Drei von ihnen seien im Krankenhaus behandelt worden. Ein Pilot eines Polizeihubschraubers erlitt den Angaben zufolge Augenverletzungen durch Laserpointer. Auch zahlreiche Demonstranten wurden verletzt - einige schwer.            Blockaden und Brände auch am Morgen            Zum Auftakt des G20-Gipfels eskaliert die Lage am Freitagmorgen erneut - Rauch steht über der Stadt.                        In der Nacht beruhigte sich die Lage, bevor die Gipfelgegner am Morgen 

In [36]:
client = Elasticsearch(hosts='robin-in.space')

In [35]:
class ArticleQueryBuilder():
    def __init__(self, article):
        self.article = article

        
    def simple_and_query(self):
        return Q("match", title=self.article['title']) & Q("match", body=self.article['body'])
        
        
    def simple_or_query(self):
        return Q("match", title=self.article['title']) | Q("match", body=self.article['body'])
    
    
    def date_filter(self, before=timedelta(weeks=2),
                            after=timedelta(weeks=8)):
        date = datetime.strptime(self.article['date'], '%Y-%m-%d')
        return Q("bool", filter=[Q('range', 
                            published={
                                'gte': date - before,
                                'lte': date + after
                            })])
    
    def location_query(self):
        return Q("bool", should=[
            Q("match", body=self.article['location']), 
            Q("match", title=self.article['location']),
            Q("match", officeName=self.article['location'])])

In [37]:
data['demonstrationen']['data'][0]['articles'][0]

{'body': 'Am Vorabend des G20-Gipfels in Hamburg ist es wie befürchtet zu massiven Auseinandersetzungen zwischen Polizei und Demonstranten gekommen. Bei der "Welcome to Hell"-Demonstration gegen das Treffen flogen am Donnerstagabend Flaschen, Feuerwerkskörper wurden gezündet, später brannten umgestürzte Mülltonnen und Autos. Die Polizei stürmte in den Demonstrationszug, setzte wiederholt Wasserwerfer und Pfefferspray ein und trieb die rund 12.000 Teilnehmer auseinander. Die Polizei meldete mindestens 111 verletzte Beamte. Drei von ihnen seien im Krankenhaus behandelt worden. Ein Pilot eines Polizeihubschraubers erlitt den Angaben zufolge Augenverletzungen durch Laserpointer. Auch zahlreiche Demonstranten wurden verletzt - einige schwer.            Blockaden und Brände auch am Morgen            Zum Auftakt des G20-Gipfels eskaliert die Lage am Freitagmorgen erneut - Rauch steht über der Stadt.                        In der Nacht beruhigte sich die Lage, bevor die Gipfelgegner am Morgen 

In [38]:
article = data['demonstrationen']['data'][0]['articles'][0]
ab= ArticleQueryBuilder(article)
query = ab.simple_and_query() & ab.location_query() & ab.date_filter()

In [40]:
s = Search(using=client)
res = s.query(query).execute()

In [41]:
def construct_queries(article):
    builder = ArticleQueryBuilder(article)
    return [
        builder.simple_and_query(),
        builder.simple_or_query(),
        builder.simple_and_query() & builder.date_filter(),
        builder.simple_or_query() & builder.date_filter(),
        builder.simple_and_query() & builder.location_query(),
        builder.simple_or_query() & builder.location_query(),
        builder.simple_and_query() | builder.location_query(),
        builder.simple_or_query() | builder.location_query(),
        builder.simple_and_query() & builder.location_query() & builder.date_filter(),
        builder.simple_or_query() & builder.location_query() & builder.date_filter(),
        (builder.simple_and_query() | builder.location_query()) & builder.date_filter(),
        (builder.simple_or_query() | builder.location_query()) & builder.date_filter()
    ]

In [42]:
def eval_query(query, pr_ids, client):
    res = Search(using=client).query(query).execute()
    return [(index, hit.meta.id) for index, hit in enumerate(res) if hit.meta.id in pr_ids]

In [43]:
def eval_event(event, client):
    results = defaultdict(list)
    for article in event['articles']:
        queries = construct_queries(article)
        for query in queries: 
            res = eval_query(query, event['pr-ids'], client)
            results[article['title']].append(res)
    return results

In [44]:
def eval_topic(topic, client):
    return [eval_event(event, client) for event in topic]

In [45]:
topic_results = {topic_name: eval_topic(topic['data'], client) for 
                 topic_name, topic in data.items()}

In [46]:
topic_results

{'demonstrationen': [defaultdict(list,
              {'G20: Verletzte nach eskalierter Demonstration': [[(0,
                  'HOe9VmUB_NHj735hLOoi'),
                 (2, 'z-W6VmUB_NHj735hUKfk')],
                [(0, 'HOe9VmUB_NHj735hLOoi'), (5, 'z-W6VmUB_NHj735hUKfk')],
                [(0, 'HOe9VmUB_NHj735hLOoi'), (2, 'z-W6VmUB_NHj735hUKfk')],
                [(0, 'HOe9VmUB_NHj735hLOoi'), (3, 'z-W6VmUB_NHj735hUKfk')],
                [(0, 'HOe9VmUB_NHj735hLOoi'),
                 (2, 'z-W6VmUB_NHj735hUKfk'),
                 (6, '4-e9VmUB_NHj735hLPck')],
                [(0, 'HOe9VmUB_NHj735hLOoi'), (4, 'z-W6VmUB_NHj735hUKfk')],
                [(0, 'HOe9VmUB_NHj735hLOoi'),
                 (2, 'z-W6VmUB_NHj735hUKfk'),
                 (8, '4-e9VmUB_NHj735hLPck')],
                [(0, 'HOe9VmUB_NHj735hLOoi'), (4, 'z-W6VmUB_NHj735hUKfk')],
                [(0, 'HOe9VmUB_NHj735hLOoi'), (2, 'z-W6VmUB_NHj735hUKfk')],
                [(0, 'HOe9VmUB_NHj735hLOoi'), (3, 'z-W6VmUB_NHj735h

In [16]:
var = 5

In [17]:
var = "hallo"

In [22]:
type(var)

str

In [21]:
var = "5"